In [9]:
class Board():
    def __init__(self, n=6, pieces=None):
        self.n = n
        self.pieces = pieces if pieces else {
            1: {1: 8, 2: 0},  # For Orange (1: Kitten, 2: Cat)
            -1: {-1: 8, -2: 0},  # For Grey (-1: Kitten, -2: Cat)
        }

        # Create the empty board array.
        self.board = [None]*self.n
        for i in range(self.n):
            self.board[i] = [0]*self.n

    # add [][] indexer syntax to the Board
    def __getitem__(self, index): 
        return self.pieces[index]

    # returns a list of coordinates of playable moves.
    def get_legal_moves(self, player):
        """Returns all the legal moves for the given color.
        (2 for Orange Cat, 1 for Orange Kitten, -1 for Grey Kitten, -2 for Grey Cat)
        """
        moves = []  # stores the legal moves.

        # Check if player is out of pieces altogether
        total_pieces = sum(self.pieces[player].values())

        # If player has no more pieces left, remove a piece from the board
        if total_pieces <= 0:  
            for y in range(self.n):
                for x in range(self.n):
                    if self.board[x][y] == player * 1:  # If there's a kitten of the same color at (x, y)
                        newmove = ((x, y), player*2)  # Promotion move, kitten is removed from the board
                        moves.append(newmove)
        else:
            # Check which pieces are available for current player
            available_pieces = [piece for piece, count in self.pieces[player].items() if count > 0]
            # For each available piece, find the legal moves
            for piece in available_pieces:
                for y in range(self.n):
                    for x in range(self.n):
                        if self.board[x][y] == 0:  # ensure the cell is empty
                            newmove = ((x, y), piece)
                            moves.append(newmove)
        return moves
    
    def execute_move(self, move, color, pieceType):
        """Perform the given move on the board; 
        color gives the color of the piece to play (1=orange, -1=grey)
        type indicates type of the piece to play (1=kitten, 2=cat)
        """

        (x,y) = move
        
        # If the cell is empty, add the piece to the empty square.
        if self.board[x][y] == 0:
            self.board[x][y] = pieceType
            self.boop(x, y)
            self.checkPromotion()
            self.pieces[color][pieceType] -= 1
            
        # If the cell contains a kitten of the same color, promote the kitten to a cat.
        elif self.board[x][y] == color:
            self.board[x][y] = 0
            self.pieces[color][2*color] += 1  # Increase cat count
            
    def boop(self, row, col):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]  # 8 directions
        current_piece = self.board[row][col]
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 6 and 0 <= new_col < 6:  # check if in bounds
                if self.board[new_row][new_col] != 0:  # check if the spot is not empty
                    # Do not move if the current piece is a kitten and the piece to be moved is a cat
                    if abs(current_piece) == 1 and abs(self.board[new_row][new_col]) == 2:
                        continue
                    new_new_row, new_new_col = new_row + dr, new_col + dc
                    if 0 <= new_new_row < 6 and 0 <= new_new_col < 6:  # check if new spot is in bounds
                        if self.board[new_new_row][new_new_col] == 0:  # check if new spot is unoccupied
                            self.board[new_new_row][new_new_col] = self.board[new_row][new_col]  # move piece
                            self.board[new_row][new_col] = 0  # empty old spot
                    else:
                        # If the new spot is out of bounds, remove the piece and update the counts
                        piece = self.board[new_row][new_col]
                        color = piece / abs(piece)  # This will give 1 for Orange and -1 for Grey
                        self.board[new_row][new_col] = 0
                        if abs(piece) == 1:
                            self.pieces[color][1 * color] += 1  # increase kitten count
                        if abs(piece) == 2:
                            self.pieces[color][2 * color] += 1  # increase cat count

    def findThreeInRowHorizontal(self):
        for row in self.board:
            for i in range(4):
                # Check if the pieces are of the same color (positive for Orange, negative for Grey)
                if (row[i] > 0 and row[i + 1] > 0 and row[i + 2] > 0) or (row[i] < 0 and row[i + 1] < 0 and row[i + 2] < 0):
                    # If all pieces are Cats, skip this row
                    if abs(row[i]) == abs(row[i + 1]) == abs(row[i + 2]) == 2:
                        continue
                    color = 1 if row[i] > 0 else -1  # get color
                    self.pieces[color][color * 2] += 3  # increment the cat count for this color
                    row[i] = row[i + 1] = row[i + 2] = 0
                    return True
        return False

    def findThreeInRowVertical(self):
        for j in range(6):
            for i in range(4):
                # Check if the pieces are of the same color (positive for Orange, negative for Grey)
                if (self.board[i][j] > 0 and self.board[i + 1][j] > 0 and self.board[i + 2][j] > 0) or \
                   (self.board[i][j] < 0 and self.board[i + 1][j] < 0 and self.board[i + 2][j] < 0):
                    # If all pieces are Cats, skip this row
                    if abs(self.board[i][j]) == abs(self.board[i + 1][j]) == abs(self.board[i + 2][j]) == 2:
                        continue
                    color = 1 if self.board[i][j] > 0 else -1  # get color
                    self.pieces[color][color * 2] += 3  # increment the cat count for this color
                    self.board[i][j] = self.board[i + 1][j] = self.board[i + 2][j] = 0
                    return True
        return False

    def findThreeInRowMainDiagonals(self):
        for i in range(4):
            for j in range(4):
                # Check if the pieces are of the same color (positive for Orange, negative for Grey)
                if (self.board[i][j] > 0 and self.board[i + 1][j + 1] > 0 and self.board[i + 2][j + 2] > 0) or \
                   (self.board[i][j] < 0 and self.board[i + 1][j + 1] < 0 and self.board[i + 2][j + 2] < 0):
                    # If all pieces are Cats, skip this row
                    if abs(self.board[i][j]) == abs(self.board[i + 1][j + 1]) == abs(self.board[i + 2][j + 2]) == 2:
                        continue
                    color = 1 if self.board[i][j] > 0 else -1  # get color
                    self.pieces[color][color * 2] += 3  # increment the cat count for this color
                    self.board[i][j] = self.board[i + 1][j + 1] = self.board[i + 2][j + 2] = 0
                    return True
        return False    

    def findThreeInRowMainAntiDiagonals(self):
        for i in range(4):
            for j in range(2, 6):
                # Check if the pieces are of the same color (positive for Orange, negative for Grey)
                if (self.board[i][j] > 0 and self.board[i + 1][j - 1] > 0 and self.board[i + 2][j - 2] > 0) or \
                   (self.board[i][j] < 0 and self.board[i + 1][j - 1] < 0 and self.board[i + 2][j - 2] < 0):
                    # If all pieces are Cats, skip this row
                    if abs(self.board[i][j]) == abs(self.board[i + 1][j - 1]) == abs(self.board[i + 2][j - 2]) == 2:
                        continue
                    color = 1 if self.board[i][j] > 0 else -1  # get color
                    self.pieces[color][color * 2] += 3  # increment the cat count for this color
                    self.board[i][j] = self.board[i + 1][j - 1] = self.board[i + 2][j - 2] = 0
                    return True
        return False    
    
    def checkPromotion(self):
        if self.findThreeInRowHorizontal() or self.findThreeInRowVertical() or self.findThreeInRowMainDiagonals() or self.findThreeInRowMainAntiDiagonals():
            return True
        else:
            return False
        
    def findThreeCatsInRowHorizontal(self, color):
        for row in self.board:
            for i in range(4):
                if row[i] == row[i + 1] == row[i + 2] == color * 2:
                    return True
        return False

    def findThreeCatsInRowVertical(self, color):
        for j in range(6):
            for i in range(4):
                if self.board[i][j] == self.board[i + 1][j] == self.board[i + 2][j] == color * 2:
                    return True
        return False

    def findThreeCatsInRowMainDiagonals(self, color):
        for i in range(4):
            for j in range(4):
                if self.board[i][j] == self.board[i + 1][j + 1] == self.board[i + 2][j + 2] == color * 2:
                    return True
        return False    

    def findThreeCatsInRowMainAntiDiagonals(self, color):
        for i in range(4):
            for j in range(2, 6):
                if self.board[i][j] == self.board[i + 1][j - 1] == self.board[i + 2][j - 2] == color * 2:
                    return True
        return False  

    def is_win(self, color):
        # Check horizontal, vertical and both diagonal directions
        if self.findThreeCatsInRowHorizontal(color) or \
            self.findThreeCatsInRowVertical(color) or \
            self.findThreeCatsInRowMainDiagonals(color) or \
            self.findThreeCatsInRowMainAntiDiagonals(color):
            return True

        # check for eight cats of the same color on the board
        flat_board = [item for sublist in self.board for item in sublist]
        if flat_board.count(color * 2) == 8:
            return True

        # no win condition satisfied
        return False

In [10]:
import numpy as np
import copy

class BoopGame():
    def __init__(self, n=6):
        self.n = n
        self.pieces = {
            1: {1: 8, 2: 0},  # For Orange (1: Kitten, 2: Cat)
            -1: {-1: 8, -2: 0},  # For Grey (-1: Kitten, -2: Cat)
        }

    def getInitBoard(self):
        # return initial board (numpy board)
        b = Board(self.n)
        return np.array(b.board)

    # Add this method to your BoopGame class
    def getInitialPieces(self):
        return {
            1: {1: 8, 2: 0},  # For Orange (1: Kitten, 2: Cat)
            -1: {-1: 8, -2: 0},  # For Grey (-1: Kitten, -2: Cat)
        }

    def getBoardSize(self):
        # (a,b) tuple
        return (self.n, self.n)

    def getActionSize(self):
        # return number of actions: 2 actions (place kitten or cat) for each square, plus 1 for pass
        return self.n*self.n*2
    
    def getNextState(self, board, player, action, pieces):
        b = Board(self.n, copy.deepcopy(pieces))
        b.board = np.copy(board)
        
        pieceType = 1 * player if action < self.n*self.n else 2 * player  # 1 for kitten, 2 for cat
        action = action % (self.n*self.n)  # Now action is within the range of the number of board positions

        move = (int(action/self.n), action%self.n)
        b.execute_move(move, player, pieceType)
        assert sum(value for key, value in b.pieces[player].items()) + np.count_nonzero(b.board == player * 1) + np.count_nonzero(b.board == player * 2) == 8, "The number of pieces on the board and off the board do not sum up to 8"
        return (b.board, -player, b.pieces)

    # 1 = Player Orange, -1 = Player Grey
    def getValidMoves(self, board, player, pieces):
        print("VALID MOVES: " + str(player))
        # return a fixed size binary vector
        valids = [0]*self.getActionSize()
        b = Board(self.n, copy.deepcopy(pieces))
        b.board = np.copy(board)
        legalMoves = b.get_legal_moves(player)
        if len(legalMoves)==0:
            return np.array(valids)

        for move in legalMoves:
            (x, y), piece = move

            if piece == player * 1:  # If it's a kitten
                index = self.n*x + y
                valids[index] = 1

            elif piece == player * 2:  # If it's a cat
                index = self.n*self.n + self.n*x + y
                valids[index] = 1

        return np.array(valids)

    def getGameEnded(self, board, player, pieces):
        # return 0 if not ended, 1 if player 1 won, -1 if player 1 lost
        b = Board(self.n, copy.deepcopy(pieces))
        b.board = np.copy(board)

        if b.is_win(player):
            return 1
        if b.is_win(-player):
            return -1

        # draw has a very little value 
        return 0

    def getCanonicalForm(self, board, player):
        # return state if player==1, else return -state if player==-1
        return player*board

    def stringRepresentation(self, board):
        return board.tostring()

    def stringRepresentationReadable(self, board):
        board_s = "".join(self.square_content[square] for row in board for square in row)
        return board_s

    def getSymmetries(self, board, pi):
        # mirror, rotational
        assert(len(pi) == 2 * self.n**2)  # No pass action
        mid_index = len(pi) // 2
        pi_board_kittens = np.reshape(pi[:mid_index], (self.n, self.n))
        pi_board_cats = np.reshape(pi[mid_index:], (self.n, self.n))
        l = []

        for i in range(1, 5):
            for j in [True, False]:
                newB = np.rot90(board, i)
                newPi_kittens = np.rot90(pi_board_kittens, i)
                newPi_cats = np.rot90(pi_board_cats, i)
                if j:
                    newB = np.fliplr(newB)
                    newPi_kittens = np.fliplr(newPi_kittens)
                    newPi_cats = np.fliplr(newPi_cats)
                newPi = np.concatenate((newPi_kittens.ravel(), newPi_cats.ravel()))
                l += [(newB, list(newPi))]
        return l

    @staticmethod
    def display(board):
        display(board)

In [11]:
import numpy as np
import random 

class RandomPlayer():
    def __init__(self, game):
        self.game = game

    def play(self, board, player, pieces):
        print("___________________")
        a = np.random.randint(self.game.getActionSize())
        valids = self.game.getValidMoves(board, player, pieces)
        while valids[a]!=1:
            a = np.random.randint(self.game.getActionSize())
        return a
    
class RandomPlayerWithCenterBias():
    def __init__(self, game):
        self.game = game

    def play(self, board, player, pieces):
        print("___________________")
        
        # Define the weights for each number
        weights = [10 if x in [14, 15, 20, 21] else 1 for x in list(range(72))]
        a = random.choices(list(range(72)), weights, k=1)[0]
        valids = self.game.getValidMoves(board, player, pieces)
        while valids[a]!=1:
            # Randomly select a number with the specified weights
            a = random.choices(list(range(72)), weights, k=1)[0]
        return a

game = BoopGame()
randomPlayer = RandomPlayer(game)
randomPlayerWithCenterBias = RandomPlayerWithCenterBias(game)
wins_and_losses = []

def playGame(maxTurns = 100):
    board = game.getInitBoard()
    pieces = game.getInitialPieces()
    player = 1
    
    for i in range(maxTurns):
        a = 0
        if player == 1:
            a = randomPlayer.play(board, player, pieces)
        if player == -1:
            a = randomPlayerWithCenterBias.play(board, player, pieces)
        board, nextplayer, pieces = game.getNextState(board, player, a, pieces)
        if game.getGameEnded(board, player, pieces) != 0:
            print("Turn: " + str(i))
            print(pieces)
            print(game.getGameEnded(board, player, pieces))
            wins_and_losses.append((player, game.getGameEnded(board, player, pieces)))
            game.display(board)
            break
        player = nextplayer
        print(pieces)
        game.display(board)
        
for i in range(1):
    playGame()
    
print("Wins/Losses: ")
print(wins_and_losses)


___________________
VALID MOVES: 1
{1: {1: 7, 2: 0}, -1: {-1: 8, -2: 0}}


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

___________________
VALID MOVES: -1
{1: {1: 7, 2: 0}, -1: {-1: 7, -2: 0}}


array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0,  0]])

___________________
VALID MOVES: 1
{1: {1: 7, 2: 0}, -1: {-1: 7, -2: 0}}


...

___________________
VALID MOVES: 1
{1: {1: 2, 2: 0}, -1: {-1: 0, -2: 1}}


array([[ 0,  0, -2,  0,  0,  0],
       [ 0,  0,  0,  0,  1, -2],
       [ 2,  0,  1, -2,  0,  0],
       [ 0,  0,  0,  1,  0,  1],
       [ 0,  0,  2,  0, -2, -2],
       [-2,  0,  0,  0,  0, -2]])

___________________
VALID MOVES: -1
Turn: 57
{1: {1: 2, 2: 0}, -1: {-1: 0, -2: 0}}
1


array([[ 0, -2,  0, -2,  0,  0],
       [ 0,  0,  0,  0,  0, -2],
       [ 2,  0,  1, -2,  0,  1],
       [ 0,  0,  0,  1,  0,  1],
       [ 0,  0,  2,  0, -2, -2],
       [-2,  0,  0,  0,  0, -2]])

Wins/Losses: 
[(-1, 1)]
